# Generador de JavaScript API

## Utilities

### Global imports

In [1]:
import django
import textwrap
import os
import json
import re

### Common functions

In [2]:
def camel2snake(string):
    regex = re.sub('(.)([A-Z][a-z]+)', r'\1_\2', string)
    return re.sub('([a-z0-9])([A-Z])', r'\1_\2', regex).lower()

In [3]:
def get_internal_type(field):
    try:
        return field.get_internal_type()
    except Exception as _:
        return field.__class__.__name__

In [4]:
def get_related_model(field):
    try:
        return field.related_model._meta.label
    except Exception as _:
        return None

## API Generator

In [5]:
def generateJsAPI(model, destination_dir):
    # Name of the model
    model_name = model.__name__.replace("Bss", "")
    
    # Lookup field in Django API
    lookup_field = getattr(model, 'lookup_field', 'id')
    # for aestethics
    lookup_type = 'int' if lookup_field == 'id' else 'string'
    lookup_type = f'{{{lookup_type}}}'
    
    file_name = camel2snake(model_name).replace('_','-')
    endpoint = camel2snake(model_name).upper()
    path_name = camel2snake(model_name).replace('_','-')
    object_name = camel2snake(model_name)
    
    api = f"""        /**
         * Shortcut fuctions to make calls to API endpoints related to {object_name.title()} object
         */
        import {{ camelToSnake }} from '@/_utils'
        import {{ paramsSerializer }} from './_utils'
        import {{ {endpoint} }} from './_endpoints'
        import session from './_session'
        
        export default {{
            /**
             * Performs options request to get {object_name}'s schema.
             */
            options (config = {{}}) {{ 
                return session.options({endpoint}, config) 
            }},
            /**
             * Performs {object_name} listing
             */
            list (query = {{}}, config = {{}}) {{
                config.params = camelToSnake(query)
                config.paramsSerializer = paramsSerializer
                return session.get({endpoint}, config)
            }},
            /**
             * Performs {object_name} creation
             *
             * @param {{object}} - {object_name.title()} data
             */
            create (data, config = {{}}) {{
                return session.post({endpoint}, data, config)
            }},
            /**
             * Performs {object_name} retrieve
             *
             * @param {{{lookup_type}}} - {object_name.title()} {lookup_field.upper()}
             */
            retrieve ({lookup_field}, query = {{}}, config = {{}}) {{
                config.params = camelToSnake(query)
                config.paramsSerializer = paramsSerializer
                return session.get(`${{{endpoint}}}${{{lookup_field}}}/`, config)
            }},
            /**
             * Performs {object_name} update
             *
             * @param {lookup_type:8s} - {object_name.title()} {lookup_field.upper()}
             * @param {{object}} - {object_name.title()} data
             */
            update ({lookup_field}, data, config = {{}}) {{
                return session.patch(`${{{endpoint}}}${{{lookup_field}}}/`, data, config)
            }},
            /**
             * Performs {object_name} retrieve
             *
             * @param {{{lookup_type}}} - {lookup_field.upper()} of {object_name.title()} to delete
             */
            delete ({lookup_field}, config = {{}}) {{
                return session.delete(`${{{endpoint}}}${{{lookup_field}}}/`, config)
            }}
        }}
        """
    
    api = textwrap.dedent(api).split('\n')
    api = [line.rstrip() for line in api]
    api = [line[:10].replace('    ', '  ') + line[10:] for line in api]
    api = '\n'.join(api)
    
    api_file = os.path.join(destination_dir, file_name + ".js")
    
    with open(api_file, 'w') as file:
        file.write(api)
    file.close()
    
    print(f"""export const {endpoint} = BASE + "{path_name}/" """)

In [6]:
models = django.apps.apps.get_models(include_auto_created=False)
models = filter(lambda m: not m._meta.abstract, models)
models = dict(map(lambda m: (m.__name__.replace("Bss", ""), m), models))
models = list(map(lambda n: (n, models[n]), sorted(models.keys())))
models

for (name, model) in models:
    # generateJsAPI(model, '../frontend/src/api/')
    generateJsAPI(model, 'autogenerated/js/api/')

export const ADDRESS = BASE + "address/" 
export const ASSOCIATION = BASE + "association/" 
export const ATTRIBUTE = BASE + "attribute/" 
export const ATTRIBUTE_TRANSLATION = BASE + "attribute-translation/" 
export const ATTRIBUTE_VALUE = BASE + "attribute-value/" 
export const ATTRIBUTE_VALUE_TRANSLATION = BASE + "attribute-value-translation/" 
export const AUTHORIZATION_KEY = BASE + "authorization-key/" 
export const BANK_ACCOUNT = BASE + "bank-account/" 
export const BANNER = BASE + "banner/" 
export const BENEFIT = BASE + "benefit/" 
export const BENEFIT_TRANSLATION = BASE + "benefit-translation/" 
export const BRAND = BASE + "brand/" 
export const BRAND_TRANSLATION = BASE + "brand-translation/" 
export const CATEGORY = BASE + "category/" 
export const CATEGORY_TRANSLATION = BASE + "category-translation/" 
export const CHECKOUT = BASE + "checkout/" 
export const CHECKOUT_LINE = BASE + "checkout-line/" 
export const CITY = BASE + "city/" 
export const CITY_AREA = BASE + "city-area/"